In [1]:
# Load Google drive where the data and models are stored
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# Load packages

import sys
import keras
import numpy as np
import pandas as pd
import random


import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import resample 
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import classification_report

In [3]:
random.seed(10)

In [4]:
df = pd.read_csv(r'/content/drive/MyDrive/DS4A/2018.csv')



In [5]:
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,0.0,NaN,0.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,1756,1754.0,-2.0,0.0,NaN,0.0,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,922,936.0,14.0,0.0,NaN,0.0,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
airports = pd.read_csv(r'/content/drive/MyDrive/DS4A/airports.csv')
#df = df.join(df, airports, on = 'ORIGIN', how = 'inner')
#df.rename(columns = {'Company':'Brand'}, inplace = True)

#airports = pd.read_csv(r'/content/drive/MyDrive/DS4A/airports.csv')

In [7]:
# so now I'll add in the lat and lon for these two airports manually
# data is from here: https://cc.bingj.com/cache.aspx?q=ear+airport+lat&d=4669505875149362&mkt=de-CH&setlang=en-GB&w=opx1lWthNa9LcgO1WUKVt3rhfhq7dL7E
n_airports = airports.shape[0] 
airports.at[n_airports, 'Name'] = 'Kearney Regional Airport'
airports.at[n_airports, 'City'] = 'Kearney'
airports.at[n_airports, 'Country'] = 'United States'
airports.at[n_airports, 'IATA'] = 'EAR'
airports.at[n_airports, 'ICAO'] = 'KEAR'
airports.at[n_airports, 'Latitude'] = 40.72694
airports.at[n_airports, 'Longitude'] = -99.00667


In [8]:
n_airports = airports.shape[0]

airports.at[n_airports, 'Name'] = 'Laughlin/Bullhead International Airport'
airports.at[n_airports, 'City'] = 'Bullhead City' # technically this in in Bullhead City, Arizona and Laughlin, Nevada
airports.at[n_airports, 'Country'] = 'United States'
airports.at[n_airports, 'IATA'] = 'IFP'
airports.at[n_airports, 'ICAO'] = 'KIFP'
airports.at[n_airports, 'Latitude'] = 35.156111
airports.at[n_airports, 'Longitude'] = -114.559444

In [9]:
df2 = pd.merge(df, airports, how='left', left_on=['ORIGIN'], right_on=['IATA'])


In [10]:
df2.rename(columns = {'Latitude': 'ORIGIN_LAT', 'Longitude': 'ORIGIN_LON'}, inplace = True)

In [11]:
df2 = pd.merge(df2, airports, how='left', left_on=['DEST'], right_on=['IATA'])
df2.rename(columns = {'Latitude': 'DEST_LAT', 'Longitude': 'DEST_LON'}, inplace = True)
df2.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,Name_x,City_x,Country_x,IATA_x,ICAO_x,ORIGIN_LAT,ORIGIN_LON,Name_y,City_y,Country_y,IATA_y,ICAO_y,DEST_LAT,DEST_LON
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN,Newark Liberty International Airport,Newark,United States,EWR,KEWR,40.692501,-74.168701,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672996
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN,McCarran International Airport,Las Vegas,United States,LAS,KLAS,36.080101,-115.152000,San Francisco International Airport,San Francisco,United States,SFO,KSFO,37.618999,-122.375000
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,0.0,NaN,0.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN,John Wayne Airport-Orange County Airport,Santa Ana,United States,SNA,KSNA,33.675701,-117.867996,Denver International Airport,Denver,United States,DEN,KDEN,39.861698,-104.672996
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,1756,1754.0,-2.0,0.0,NaN,0.0,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN,Southwest Florida International Airport,Fort Myers,United States,RSW,KRSW,26.536200,-81.755203,Chicago O'Hare International Airport,Chicago,United States,ORD,KORD,41.978600,-87.904800
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,922,936.0,14.0,0.0,NaN,0.0,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN,Chicago O'Hare International Airport,Chicago,United States,ORD,KORD,41.978600,-87.904800,Albany International Airport,Albany,United States,ALB,KALB,42.748299,-73.801697


In [12]:
df2 = df2.loc[:,~df2.columns.str.contains('_x', case=False)] 
df2= df2.loc[:,~df2.columns.str.contains('_y', case=False)] 


In [13]:
df2

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,ORIGIN_LAT,ORIGIN_LON,DEST_LAT,DEST_LON
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN,40.692501,-74.168701,39.861698,-104.672996
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN,36.080101,-115.152000,37.618999,-122.375000
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,0.0,NaN,0.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN,33.675701,-117.867996,39.861698,-104.672996
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,1748.0,6.0,1756,1754.0,-2.0,0.0,NaN,0.0,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN,26.536200,-81.755203,41.978600,-87.904800
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,926.0,10.0,922,936.0,14.0,0.0,NaN,0.0,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN,41.978600,-87.904800,42.748299,-73.801697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7213441,2018-12-31,AA,1815,DCA,CLT,1534,1530.0,-4.0,20.0,1550.0,1702.0,7.0,1714,1709.0,-5.0,0.0,NaN,0.0,100.0,99.0,72.0,331.0,NaN,NaN,NaN,NaN,NaN,NaN,38.852100,-77.037697,35.214001,-80.943100
7213442,2018-12-31,AA,1816,CLT,DFW,1751,1757.0,6.0,18.0,1815.0,1943.0,10.0,1952,1953.0,1.0,0.0,NaN,0.0,181.0,176.0,148.0,936.0,NaN,NaN,NaN,NaN,NaN,NaN,35.214001,-80.943100,32.896801,-97.038002
7213443,2018-12-31,AA,1817,CLT,MEM,2015,2010.0,-5.0,36.0,2046.0,2114.0,4.0,2107,2118.0,11.0,0.0,NaN,0.0,112.0,128.0,88.0,511.0,NaN,NaN,NaN,NaN,NaN,NaN,35.214001,-80.943100,35.042400,-89.976700
7213444,2018-12-31,AA,1818,CLT,RDU,1300,1323.0,23.0,11.0,1334.0,1400.0,4.0,1350,1404.0,14.0,0.0,NaN,0.0,50.0,41.0,26.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN,35.214001,-80.943100,35.877602,-78.787498


In [14]:
test = df2[df2.ORIGIN_LAT.isna()]
np.unique(test.ORIGIN)

array([], dtype=object)

In [ ]:
#departure delays by airport (there are more than 5K airports in the US)
d_delays_airport = df[['ORIGIN','DEP_DELAY']].groupby(['ORIGIN'])
result = [g[1] for g in list(d_delays_airport))[:3]]
#d_delays_airport.plot.bar()
plt = result.boxplot(subplots=False, rot=45, fontsize=12)
#plt.xlabel('Airport')
#plt.ylabel('# of Delayed Flights')
#plt.title('Delayed Flights by Airport - 2018')
plt

SyntaxError: ignored

In [ ]:
# concatenate years
# frames = [df1, df2, df3]
# result = pd.concat(frames)
dataa = df

In [ ]:
dataa.shape

(7213446, 28)

In [ ]:
 
#dataa = pd.merge(df_airports, airports, on='airport')

In [ ]:
# we have to make the date numeric to be fed into the NN
dataa['FL_DATE'] = pd.DatetimeIndex(dataa['FL_DATE']).month
# geocode airports
dataa = dataa.select_dtypes(['number'])

In [ ]:
dataa['DELAY_FLAG'] = dataa['DEP_DELAY'].gt(15)
dataa['DELAY_FLAG']=dataa['DELAY_FLAG'].astype('uint8')

#dataa['DELAY_FLAG'] = dataa[dataa['DEP_DELAY'] >= 1] = 1

#dataa['DELAY_FLAG'] = dataa[is.na(dataa['DEP_DELAY'])] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
dataa.shape

(7213446, 25)

In [ ]:
(dataa['DELAY_FLAG'] == 0).sum()/7213446

In [ ]:
dataa.DELAY_FLAG.unique()

array([0, 1], dtype=uint8)

In [ ]:
dataa=pd.DataFrame(dataa).fillna(value = -9999999)

In [ ]:
from sklearn.model_selection import StratifiedKFold
#create definitions to prepare the data, splitt the data in train and test and scale the data
def load_dataset():
    dataset = dataa
    X = dataset.iloc[:, 0:23].values
    y = dataset.iloc[:, 24].values
    return X, y
def prepare_train_test_set(X, y):
    skf=StratifiedKFold(n_splits=2)#, random_state=None, shuffle=False)
    for train_index, test_index in skf.split(X, y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X[train_index], X[test_index]
      y_train, y_test = y[train_index], y[test_index]
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    return X_train, X_test, y_train, y_test
def scale_features(X_train, X_test):
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    return X_train, X_test
#run the data preparation
if __name__ == '__main__':
    labelencoder_x_1 = LabelEncoder()
    dataa.iloc[:, 0] = labelencoder_x_1.fit_transform(dataa.iloc[:, 0])
    dataset_path = sys.argv[1]
    X, y = load_dataset()
    X_train, X_test, y_train, y_test = prepare_train_test_set(X, y)
    X_train, X_test = scale_features(X_train, X_test)

TRAIN: [3601258 3601259 3601260 ... 7213443 7213444 7213445] TEST: [      0       1       2 ... 3635380 3635381 3635386]
TRAIN: [      0       1       2 ... 3635380 3635381 3635386] TEST: [3601258 3601259 3601260 ... 7213443 7213444 7213445]


In [ ]:
#Prepare the earlystop and saved weights
filepath = "model_weights.hdf5"
callbbacks_list = [ModelCheckpoint(filepath, monitor = "val_loss", save_best_only = True, mode = "min")]

In [ ]:
nodes_list2 = [1]
roc_outcomes = []
pr_outcomes = []
loss_hx_list = []
model_number = np.array(np.arange(1, len(nodes_list2)+1)).tolist()
count = 0
columns_list = ["Epochs"]

#Run neural network for all nodes 
import time 
start_time = time.time()

for nodes in nodes_list2:
  count += 1


  columns_list.append("model " + str(count))

  print("\n")
  print("model " + str(count) + "/" + str(len(nodes_list2)))
  print("\n")

  model = Sequential()
  model.add(Dense(units = 23,input_dim=23,activation='relu'))
  model.add(Dense(units = nodes, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
  model.fit(X_train, y_train, validation_split = 0.25, epochs=1, batch_size = 32, callbacks = [EarlyStopping(monitor="val_loss", patience=3)])

  #pred_prob = model.predict_prob(X_test)
  #predictions = model.predict_classes(X_test) #this is the original code
  pred_prob=model.predict(X_test)
  #predictions=np.argmax(predict_prob,axis=1)
  predictions = (model.predict(X_test) > 0.5).astype("int32")

  logit_roc_out = roc_auc_score(y_test, pred_prob)
  average_precision = average_precision_score(y_test, predictions)
  roc_outcomes.append(logit_roc_out)
  pr_outcomes.append(average_precision)

  cm = pd.DataFrame(confusion_matrix(y_test, predictions))
  cm["Total"] = np.sum(cm, axis =1)
  cm = cm.append(np.sum(cm, axis =0), ignore_index =True)
  cm.columns = ["Predicted No", "Predicted Yes", "Total"]
  cm = cm.set_index([["Actual No", "Actual Yes", "Total" ]])
  print("Model Confusion Matrix")
  print(cm)

  print("Model Classification Report")
  print(classification_report(y_test, predictions))

  elapsed_time = (time.time() - start_time)/60
  pct_comp = (count/len(nodes_list2))*100
  print(str(round(pct_comp, 2)) + "% complete" + " (Elapsed time = " + str(round(elapsed_time, 2)) + " min")




model 1/1


84533/84533 [==============================] - 163s 2ms/step - loss: 0.2360 - binary_accuracy: 0.9244 - val_loss: 0.2448 - val_binary_accuracy: 0.9214
Model Confusion Matrix
            Predicted No  Predicted Yes    Total
Actual No        2808574         166738  2975312
Actual Yes        109984         521427   631411
Total            2918558         688165  3606723
Model Classification Report
              precision    recall  f1-score   support

           0       0.96      0.94      0.95   2975312
           1       0.76      0.83      0.79    631411

    accuracy                           0.92   3606723
   macro avg       0.86      0.88      0.87   3606723
weighted avg       0.93      0.92      0.92   3606723

100.0% complete (Elapsed time = 6.42 min


In [ ]:
#lf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                    hidden_layer_sizes=(5, 2), random_state=1)

#clf.fit(X, y)

In [ ]:
#plot network accuracy vs epoch